# Workflow to compare TISSUES and NextProt data

### Import the required packages

In [31]:
## based on the tutorial https://jensenlab.org/training/stringapp/#exercise-3

import os
import sys
from time import sleep
import pandas as pd
import matplotlib
from py2cytoscape import cyrest
from IPython.display import Image
%matplotlib inline

### Setup Cytoscape
TODO: check if apps are installed and if not, install them

In [32]:
cytoscape=cyrest.cyclient()
cytoscape.version()

apiVersion v1
cytoscapeVersion 3.7.2


In [33]:
cytoscape.session.new()

### Protein network retrieval
Retrieve a STRING network for all UniProt identifiers in the table. 

In [34]:
from py2cytoscape.cyrest.base import api
## create the string network with the uniprot ids
api(namespace="string",command="pubmed query",PARAMS={"cutoff": "0.4",
                                                      "limit": "20",
                                                      "pubmed": "aging",
                                                      "species": "homo sapiens",
                                                      "taxonID": "9606"})
sleep(2)
fig=cytoscape.networks.getFirstImageAsPng(networkId=cytoscape.network.get()["SUID"],h=None)
Image(fig.content)

### Check for obsolete UniProt IDs

In [35]:
## get list 
uniprotIDs=list(set(cytoscape.table.getTable(columns=['stringdb::canonical name'],table='node')['stringdb::canonical name']))

### for python 2

In [36]:
import urllib,urllib2
import pandas as pd

url = "https://www.uniprot.org/uploadlists/"
params = {
"from":"ACC",
"to":"ACC",
"format":"tab",
"query":" ".join(uniprotIDs)
}
data = urllib.urlencode(params)
request = urllib2.Request(url, data)
contact = "" # Please set a contact email address here to help us debug in case of problems (see https://www.uniprot.org/help/privacy).
request.add_header("User-Agent", "Python %s" % contact)
response = urllib2.urlopen(request)

ModuleNotFoundError: No module named 'urllib2'

### for python 3

In [45]:

## convert from STRING UniProt IDs to current IDs / check for obsolete IDs
import urllib.request, urllib.parse, urllib.error,urllib.request,urllib.error,urllib.parse
url = "https://www.uniprot.org/uploadlists/"
params = {
"from":"ACC",
"to":"ACC",
"format":"tab",
"query":" ".join(uniprotIDs)
}
data = urllib.parse.urlencode(params).encode("utf-8")
request = urllib.request.Request(url, data)
contact = "" # Please set a contact email address here to help us debug in case of problems (see https://www.uniprot.org/help/privacy).
request.add_header("User-Agent", "Python %s" % contact)
response = urllib.request.urlopen(request)

In [46]:
uniprot_mapping = pd.read_csv(response, sep ="\t")
nexProt_list=["entry:NX_"+each for each in uniprot_mapping['To']]
nexProt_list

['entry:NX_Q96EB6',
 'entry:NX_Q9UEF7',
 'entry:NX_P23560',
 'entry:NX_Q9NRC8',
 'entry:NX_P04637',
 'entry:NX_P05019',
 'entry:NX_Q8N6T7',
 'entry:NX_Q9NTG7',
 'entry:NX_Q9NXA8',
 'entry:NX_P02649',
 'entry:NX_Q9Y6E7',
 'entry:NX_P01308',
 'entry:NX_P49768',
 'entry:NX_P55008',
 'entry:NX_P05067',
 'entry:NX_P02741',
 'entry:NX_P10636',
 'entry:NX_Q8IXJ6',
 'entry:NX_P05231',
 'entry:NX_P31749']

### Get NextProt data
here we will retrieve the nextProt data from SparQL query.

In [50]:
a="""PREFIX : <http://nextprot.org/rdf#>
PREFIX entry: <http://nextprot.org/rdf/entry/>
PREFIX isoform: <http://nextprot.org/rdf/isoform/>
PREFIX annotation: <http://nextprot.org/rdf/annotation/>
PREFIX evidence: <http://nextprot.org/rdf/evidence/>
PREFIX xref: <http://nextprot.org/rdf/xref/>
PREFIX publication: <http://nextprot.org/rdf/publication/>
PREFIX identifier: <http://nextprot.org/rdf/identifier/>
PREFIX cv: <http://nextprot.org/rdf/terminology/>
PREFIX gene: <http://nextprot.org/rdf/gene/>
PREFIX source: <http://nextprot.org/rdf/source/>
PREFIX db: <http://nextprot.org/rdf/db/>
PREFIX context: <http://nextprot.org/rdf/context/>
PREFIX interaction: <http://nextprot.org/rdf/interaction/>
PREFIX uniprot: <http://purl.uniprot.org/uniprot/>
PREFIX proteoform: <http://nextprot.org/rdf/proteoform/>
select distinct ?entry str(?gen) ?PAset (count(distinct(?pm)) as ?pepcnt) where {
values ?entry {"""

b=" ".join(nexProt_list)
c="""}
?entry :isoform ?iso .
  ?entry :gene / :name ?gen .
  ?iso :swissprotDisplayed true .   #restricts to canonical iso
  ?iso :peptideMapping ?pm .
  ?pm :proteotypic true .             #only uniquely mapping peptides
?pm :evidence / :assignedBy ?source .
    ?source rdfs:comment ?sourcename.
    bind(substr(str(?sourcename),48) as ?PAset)
  filter(regex(str(?sourcename),"PeptideAtlas"))   
  #filter (!regex(str(?sourcename), "Cancer"))   
  filter (!regex(str(?sourcename), "phosphoproteome")) 
  ?source rdfs:comment ?srcname.
  }
order by ?entry"""
query=a+b+c

### for python 2

In [51]:
import urllib,urllib2
import pandas as pd

url = "https://sparql.nextprot.org/"

params = {
"query":query,
"format":"csv"
}

data = urllib.urlencode(params)
request = urllib2.Request(url, data)
contact = "" # Please set a contact email address here to help us debug in case of problems (see https://www.uniprot.org/help/privacy).
request.add_header("User-Agent", "Python %s" % contact)
response = urllib2.urlopen(request)



ModuleNotFoundError: No module named 'urllib2'

### for python 3

In [52]:
import urllib.request, urllib.parse, urllib.error,urllib.request,urllib.error,urllib.parse

import pandas as pd
from io import StringIO 

url = "https://sparql.nextprot.org/"

params = {
"query":query,
"format":"csv"
}

data = urllib.parse.urlencode(params).encode("utf-8")
request = urllib.request.Request(url, data)
contact = "" # Please set a contact email address here to help us debug in case of problems (see https://www.uniprot.org/help/privacy).
request.add_header("User-Agent", "Python %s" % contact)
response = urllib.request.urlopen(request)

In [53]:
nextProt_output = pd.read_csv(response, sep =",")
nextProt_output.head()

entry callret-1                  PAset  \
0  http://nextprot.org/rdf/entry/NX_P01308       INS           Blood Plasma   
1  http://nextprot.org/rdf/entry/NX_P01308       INS               Pancreas   
2  http://nextprot.org/rdf/entry/NX_P01308       INS            Blood Cells   
3  http://nextprot.org/rdf/entry/NX_P01308       INS       Cancer, Pancreas   
4  http://nextprot.org/rdf/entry/NX_P01308       INS  Non Cancer Cell Lines   

   pepcnt  
0       3  
1       5  
2       1  
3       2  
4       2

## Convert NextProt data and merge with TISSUES
Create an OmicsVisualizer table

### process data from sparql query

In [54]:
#EXPECTS nextprot_output df

# unstack data to get tissues as separate columns
unstacked = nextProt_output.set_index(['entry','PAset'])[['pepcnt']].unstack()

# add gene names back to unstacked data
genes=nextProt_output.set_index('entry')['callret-1']
unstacked.reset_index(inplace=True)
mapped_genes=unstacked['entry'].map(genes.to_dict())
unstacked.insert(0,'gene',mapped_genes)
unstacked.set_index("entry")

gene        pepcnt  \
PAset                                          Adrenal Gland   
entry                                                          
http://nextprot.org/rdf/entry/NX_P01308    INS           NaN   
http://nextprot.org/rdf/entry/NX_P02649   APOE          23.0   
http://nextprot.org/rdf/entry/NX_P02741    CRP           9.0   
http://nextprot.org/rdf/entry/NX_P04637   TP53           NaN   
http://nextprot.org/rdf/entry/NX_P05019   IGF1           NaN   
http://nextprot.org/rdf/entry/NX_P05067    APP          12.0   
http://nextprot.org/rdf/entry/NX_P05231    IL6           NaN   
http://nextprot.org/rdf/entry/NX_P10636   MAPT           3.0   
http://nextprot.org/rdf/entry/NX_P23560   BDNF           NaN   
http://nextprot.org/rdf/entry/NX_P31749   AKT1          12.0   
http://nextprot.org/rdf/entry/NX_P49768  PSEN1           3.0   
http://nextprot.org/rdf/entry/NX_P55008   AIF1           1.0   
http://nextprot.org/rdf/entry/NX_Q8IXJ6  SIRT2           5.0   
http://nextprot.org/rdf/entry/NX_Q8N6T7  SIRT6           NaN   
http://nextprot.org/rdf/entry/NX_Q96EB6  SIRT1           NaN   
http://nextprot.org/rdf/entry/NX_Q9NRC8  SIRT7           NaN   
http://nextprot.org/rdf/entry/NX_Q9NTG7  SIRT3           3.0   
http://nextprot.org/rdf/entry/NX_Q9NXA8  SIRT5           8.0   
http://nextprot.org/rdf/entry/NX_Q9UEF7     KL           NaN   
http://nextprot.org/rdf/entry/NX_Q9Y6E7  SIRT4           NaN   

                                                                             \
PAset                                   Alimentary System Blood Blood Cells   
entry                                                                         
http://nextprot.org/rdf/entry/NX_P01308               NaN   NaN         1.0   
http://nextprot.org/rdf/entry/NX_P02649              30.0  22.0        95.0   
http://nextprot.org/rdf/entry/NX_P02741              10.0   NaN         NaN   
http://nextprot.org/rdf/entry/NX_P04637               NaN  13.0         1.0   
http://nextprot.org/rdf/entry/NX_P05019               NaN   1.0         4.0   
http://nextprot.org/rdf/entry/NX_P05067              19.0  27.0        47.0   
http://nextprot.org/rdf/entry/NX_P05231               NaN   NaN        20.0   
http://nextprot.org/rdf/entry/NX_P10636               8.0  25.0         3.0   
http://nextprot.org/rdf/entry/NX_P23560               NaN   NaN         4.0   
http://nextprot.org/rdf/entry/NX_P31749              16.0  22.0        26.0   
http://nextprot.org/rdf/entry/NX_P49768               2.0   9.0         6.0   
http://nextprot.org/rdf/entry/NX_P55008              13.0   9.0        23.0   
http://nextprot.org/rdf/entry/NX_Q8IXJ6              14.0  24.0        21.0   
http://nextprot.org/rdf/entry/NX_Q8N6T7               1.0  18.0         7.0   
http://nextprot.org/rdf/entry/NX_Q96EB6               NaN  38.0        19.0   
http://nextprot.org/rdf/entry/NX_Q9NRC8               NaN   7.0        12.0   
http://nextprot.org/rdf/entry/NX_Q9NTG7               6.0   7.0        12.0   
http://nextprot.org/rdf/entry/NX_Q9NXA8              16.0  13.0        16.0   
http://nextprot.org/rdf/entry/NX_Q9UEF7               NaN   NaN         NaN   
http://nextprot.org/rdf/entry/NX_Q9Y6E7               NaN   NaN         NaN   

                                                             \
PAset                                   Blood Plasma  Brain   
entry                                                         
http://nextprot.org/rdf/entry/NX_P01308          3.0    NaN   
http://nextprot.org/rdf/entry/NX_P02649        186.0   38.0   
http://nextprot.org/rdf/entry/NX_P02741        107.0    4.0   
http://nextprot.org/rdf/entry/NX_P04637          1.0    NaN   
http://nextprot.org/rdf/entry/NX_P05019         33.0    1.0   
http://nextprot.org/rdf/entry/NX_P05067         47.0   41.0   
http://nextprot.org/rdf/entry/NX_P05231          NaN    NaN   
http://nextprot.org/rdf/entry/NX_P10636          9.0  136.0   
http://nextprot.org/rdf/entry/NX_P23560          2.0    Na

**process nextprot entry to get uniprot id**

In [55]:
def process_id(entry):
    return entry.strip().split('_')[-1]
unstacked.reset_index(inplace=True)
unstacked.insert(0,'uniprot_id',unstacked['entry'].apply(process_id))

**fill empty values with 0, assign proper column names**

In [56]:
result = unstacked.set_index('uniprot_id').fillna(0)
columns = list(result.columns.droplevel(level=0))
columns[0]='entry'
columns[1]='gene'
result.columns = columns

### Process node table

In [57]:
# EXPECTS nodes DATAFRAME
nodes = cytoscape.table.getTable(table='node', columns=['stringdb::canonical name','tissue::adrenal gland',
                                                'tissue::blood','tissue::bone','tissue::bone marrow','tissue::eye', 'tissue::gall bladder',
                                                'tissue::heart', 'tissue::intestine', 'tissue::kidney','tissue::liver','tissue::lung',
                                                'tissue::muscle','tissue::nervous system','tissue::pancreas','tissue::saliva',
                                                'tissue::skin', 'tissue::spleen', 'tissue::stomach','tissue::thyroid gland', 'tissue::urine'])

In [58]:
# find all tissues column in nodes table
nodes_tissues = [col for col in nodes.columns if 'tissue::' in col]
nodes = nodes.set_index('query term')

# select tissue columns from node table
tissue_data = nodes.loc[:,nodes_tissues]
tissue_data.head()

KeyError: 'query term'

#### get tissue mapping between nextprot and tissuedb

In [45]:
# dictionary mapping nexprot tissues to TISSUESdb tissues
tissue_mapping = {
    'Adrenal Gland':'tissue::adrenal gland',
    'Blood Cells':'tissue::blood',
    'Blood Plasma':'tissue::blood',
    'Blood':'tissue::blood',
    'Eye':'tissue::eye',
    'Heart':'tissue::heart',
    'Kidney':'tissue::kidney',
    'Liver':'tissue::liver',
    'Lung':'tissue::lung',
    'Brain':'tissue::nervous system',
    'Cerebrospinal Fluid':'tissue::nervous system',
    'Olfactory System':'tissue::nervous system',
    'Pituitary Gland':'tissue::nervous system',
    'Pancreas':'tissue::pancreas',
    'Spleen':'tissue::spleen',
    'Thyroid':'tissue::thyroid gland',
    'Urinary Bladder':'tissue::urine',
    'Ureter':'tissue::urine',
    'Urine':'tissue::urine',
    'Alimentary System':'tissue::intestine',
}

In [46]:
# create dict with list of nextprot tissues belonging to each tissues tissue
tissues_to_np = {}

for key,value in tissue_mapping.items():
    if value in tissues_to_np.keys():
        tissues_to_np[value].append(key)
    else:
        tissues_to_np[value] = [key]

In [48]:
# stack tissues data into the required format
stacked_tissues = pd.DataFrame(tissue_data.stack())
stacked_tissues.columns=['tissues score']
stacked_tissues.head()

### determine peptide evidence for each mapped tissue from nextprot

In [ ]:
next_prot_bools = []

# go over all tissues in nodes data
for ttissue in nodes_tissues:
    # skip tissue if it doesnt map to nextprot
    if not ttissue in tissues_to_np.keys():
        continue
    # get nexprot tissues mapping to this tissue
    np_tissues = tissues_to_np[ttissue]
    # get peptide counts for all these tissues
    counts = result.loc[:,np_tissues]
    # boolean for peptide presence or absence
    summed = counts.sum(axis=1)
    bools = pd.DataFrame(summed!= 0)
    bools.columns = [ttissue]
    # add dataframe to list
    next_prot_bools.append(bools)
    
# concatenate the dataframes with nextprot boolean arrays
nextprot_result = pd.concat(next_prot_bools, axis =1) 

# stack the datataframe to be in the 
# same format as the stacked tissues data
nextprot_stacked = pd.DataFrame(nextprot_result.stack())
nextprot_stacked.reset_index(inplace=True)
nextprot_stacked.columns = ['uniprot_id','tissue','np']
nextprot_stacked.head()

#### merge stacked tissues and stacked nextprot dataframes

In [ ]:
merged_data = stacked_tissues.merge(
    nextprot_stacked,
    how='inner',
    left_index=True,
    right_on=['uniprot_id','tissue'])
merged_data = merged_data[['uniprot_id','tissue','tissues score','np']]

# strip the '::tissue' part from the tissue names
def strip_tissue(value):
    return value.split('::')[-1]
merged_data['tissue'] = merged_data['tissue'].apply(strip_tissue)
merged_data.head()


In [ ]:
merged_data.to_csv(
    './data/stacked_tissues_nextprot.tsv',
    index=False,
    sep='\t')

## Save the session

In [27]:
cytoscape.session.save_as(session_file="tissues_nextprot_merge")